In [24]:
import pandas as pd
import re

In [25]:
# 섹션 분리 - 2, 3만 사용
def tidy_sentences(section):
    sentence_enders = re.compile(r'((?<=[함음됨임봄짐움])(\s*\n|\.|;)|(?<=다)\.)\s*')
    # 함음됨임봄짐움 으로 끝나고, (공백과 개행문자 혹은 . 혹은 ;로 끝난다) 혹은 ('다'로 끝나고 .) 공백)인 경우
    splits = list((m.start(), m.end()) for m in re.finditer(sentence_enders, section))
    starts = [0] + [i[1] for i in splits]
    ends = [i[0] for i in splits]
    sentences = [section[start:end] for start, end in zip(starts[:-1], ends)]
    for i, s in enumerate(sentences):
        sentences[i] = (s.replace('\n', ' ').replace(' ', ' ')) + '.'

    text = '\n'.join(sentences) if len(sentences) > 0 else ''
    return sentences, text

In [26]:
import re 

def preprocess_minutes(minutes):

    pos = re.search('(.?국내외\s?경제\s?동향.?과 관련하여,?|\(가\).+경제전망.*|\(가\) 국내외 경제동향 및 평가)\n?\s*일부 위원은', minutes, re.MULTILINE)
    s1 = pos.start() if pos else -1
    pos = re.search('(.?외환.?국제금융\s?동향.?과 관련하여.*|\(나\) 외환.국제금융\s?(및 금융시장)?\s?동향)\n?\s*(일부 위원은|대부분의 위원들은)', minutes,re.MULTILINE)
    s2 = pos.start() if pos else -1
    pos = re.search('(.?금융시장\s?동향.?과 관련하여,?|\(다\) 금융시장\s?동향)\n?\s*일부 위원은', minutes, re.MULTILINE)
    s3 = pos.start() if pos else -1
    pos = re.search('((\((다|라)\) )?.?통화정책\s?방향.?에 관한 토론,?|이상과 같은 의견\s?교환을 바탕으로.*통화정책\s?방향.*에.*토론.*)\n?', minutes,re.MULTILINE)
    s4 = pos.start() if pos else -1
    pos = re.search('(\(4\) 정부측 열석자 발언.*)\n?', minutes, re.MULTILINE)
    s5 = pos.start() if pos else -1
    pos = re.search('(\(.*\) 한국은행 기준금리 결정에 관한 위원별 의견\s?개진|이상과 같은 토론에 이어 .* 관한 위원별 의견개진이 있었음.*)\n?', minutes,re.MULTILINE)
    s6 = pos.start() if pos else -1
    positer = re.finditer('(\(\s?.*\s?\) ()(심의결과|토의결론))\n?', minutes, re.MULTILINE)
    s7 = [pos.start() for pos in positer if pos.start() > s6]
    s7 = s7[0] if s7 else -1

    # 국내외 경제동향
    bos = s1
    eos = s2
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section1, section1_txt = tidy_sentences(section) #####

    # 외환․국제금융 동향
    bos = s2
    eos = s3 if s3 >= 0 else s4
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section2, section2_txt = tidy_sentences(section) #####
    #print(section)

    # 금융시장 동향
    bos = s3
    eos = s4
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section3, section3_txt = tidy_sentences(section) #####

    # 통화정책방향
    bos = s4
    eos = s5 if s5 >= 0 else s6 if s6 >= 0 else s7
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section4, section4_txt = tidy_sentences(section) #####

    # 위원별 의견 개진
    bos = s6
    eos = s7
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section5, section5_txt = tidy_sentences(section) #####

    # 정부측 열석자 발언
    bos = s5
    eos = s6
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('정부측 열석자 발언', section, re.MULTILINE)
    bos = pos.end() + 1 if pos else -1
    section = section[bos:] if bos >= 0 else section
    section6, section6_txt = tidy_sentences(section) #####

    sections = ['Economic Situation', 'Foreign Currency', 'Financial Markets',
                'Monetary Policy', 'Participants’ Views', 'Government’s View']
    section_texts = (section1, section2, section3, section4, section5, section6)


    return sections, section_texts

In [27]:
df = pd.read_csv('../data/bok_data.csv')

In [28]:
zero_list = []
for idx,i in enumerate(df.content):
    if len(preprocess_minutes(df.content[idx])[1][0]) + len(preprocess_minutes(df.content[idx])[1][1]) +\
        len(preprocess_minutes(df.content[idx])[1][2]) + len(preprocess_minutes(df.content[idx])[1][3]) +\
        len(preprocess_minutes(df.content[idx])[1][4]) + len(preprocess_minutes(df.content[idx])[1][5]) == 0:
        zero_list.append(idx)

In [29]:
zero_title_list = []
for i in zero_list:
    zero_title_list.append(df.title[i])

In [30]:
zero_title_list 
# preprocess_minutes함수 적용 결과 빈 리스트가 리턴되는 경우, 위 6개의 카테고리의 시작부분에 해당하는 내용을 언급하지 않았다.
# ex) 금융시장 동향과 관련하여~

['제8차 금통위 의사록 (4)',
 '제16차 금통위 의사록 (3)',
 '제18차 금통위 의사록 (6)',
 '제22차 금통위 의사록 (3)',
 '제26차 금통위 의사록 (1)',
 '제28차 금통위 의사록',
 '제29차 금통위 의사록',
 '제30차 금통위 의사록',
 '제2차 금통위 의사록 (3)',
 '제6차 금통위 의사록 (5)',
 '제12차 금통위 의사록 (4)',
 '제14차 금통위 의사록 (4)',
 '제18차 금통위 의사록 (5)',
 '제20차 금통위 의사록 (4)',
 '제25차 금통위 의사록 (4)',
 '제4차 금통위 의사록 (2)',
 '제6차 금통위 의사록 (4)',
 '제8차 금통위 의사록 (3)',
 '제12차 금통위 의사록 (3)',
 '제14차 금통위 의사록 (3)',
 '제18차 금통위 의사록 (4)',
 '제20차 금통위 의사록 (3)',
 '제24차 금통위 의사록 (4)',
 '제6차 금통위 의사록 (3)',
 '제8차 금통위 의사록 (2)',
 '제14차 금통위 의사록 (2)',
 '제18차 금통위 의사록 (3)',
 '제20차 금통위 의사록 (2)',
 '제24차 금통위 의사록 (3)',
 '제2차 금통위 의사록 (2)',
 '제6차 금통위 의사록 (2)',
 '제8차 금통위 의사록 (1)',
 '제12차 금통위 의사록 (2)',
 '제14차 금통위 의사록 (1)',
 '제16차 금통위 의사록 (2)',
 '제18차 금통위 의사록 (2)',
 '제20차 금통위 의사록 (1)',
 '제22차 금통위 의사록 (2)',
 '제24차 금통위 의사록 (2)',
 '제25차 금통위 의사록 (3)',
 '제2차 금통위 의사록 (1)',
 '제4차 금통위 의사록 (1)',
 '제6차 금통위 의사록 (1)',
 '제12차 금통위 의사록 (1)',
 '제13차 금통위 의사록 (1)',
 '제19차 금통위 의사록 (1)',
 '제21차 금통위 의사록 (1)',
 '제23차 금통위 의사록 (1)',
 '제25차 금통위 의사록 (2)

### df의 content 열을 2,3 섹션만 남기기

In [31]:
df['processed_content'] = 0

for idx in range(df.shape[0]):
    df.processed_content[idx] = preprocess_minutes(df.content[idx])[1]
df

/var/folders/85/178rxd5d5b31szqc8b3mznd40000gn/T/ipykernel_1161/157969725.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.processed_content[idx] = preprocess_minutes(df.content[idx])[1]
/Users/nirvana/Library/Python/3.9/lib/python/site-packages/pandas/core/dtypes/cast.py:948: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  element = np.asarray(element)


,date,title,content,processed_content
0,2011-03-10,제7차 금통위 의사록 (6),\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,([일부 위원은 과거 사례를 보면 물가가 상승하｢ ｣ 면 인플레이션 기대심리가 상승...
1,2011-03-24,제8차 금통위 의사록 (4),\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,"([], [], [], [], [], [])"
2,2011-04-12,제9차 금통위 의사록 (5),\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,([일부 위원은 과거 유가 충격 발생시 성장은 하 방리스크가 물가는 상방리스크가 ...
3,2011-05-13,제13차 금통위 의사록 (6),\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,([일부 위원은 월 전년동월대비 상승률이4 CPI｢ ｣ 월에 비해 다소 낮아진 것은...
4,2011-06-10,제15차 금통위 의사록 (5),\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,([일부 위원은 국제유가의 상승으로 에너지 자｢ ｣ 원개발 및 확보 대체에너지 개발...
...,...,...,...,...
225,2022-08-25,금융통화위원회 의사록(2022년도 제16차),\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,([일부 위원은 우리나라와 미국에서 주거비 변동이 물가에 어떠한 영향을 미치 고 ...
226,2022-09-08,금융통화위원회 의사록(2022년도 제17차),\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,"([], [], [], [], [], [])"
227,2022-09-22,금융통화위원회 의사록(2022년도 제18차),\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,"([], [], [], [], [], [])"
228,2022-10-12,금융통화위원회 의사록(2022년도 제19차),\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,"([일부 위원은 경제전망 시 금리지표로 어떠한 시장금리를 사용하는지, 그리고 금번..."


In [32]:
df = df.drop('content', axis=1).rename(columns={'processed_content':'content'})

In [34]:
df.drop('title', axis=1)

,date,content
0,2011-03-10,([일부 위원은 과거 사례를 보면 물가가 상승하｢ ｣ 면 인플레이션 기대심리가 상승...
1,2011-03-24,"([], [], [], [], [], [])"
2,2011-04-12,([일부 위원은 과거 유가 충격 발생시 성장은 하 방리스크가 물가는 상방리스크가 ...
3,2011-05-13,([일부 위원은 월 전년동월대비 상승률이4 CPI｢ ｣ 월에 비해 다소 낮아진 것은...
4,2011-06-10,([일부 위원은 국제유가의 상승으로 에너지 자｢ ｣ 원개발 및 확보 대체에너지 개발...
...,...,...
225,2022-08-25,([일부 위원은 우리나라와 미국에서 주거비 변동이 물가에 어떠한 영향을 미치 고 ...
226,2022-09-08,"([], [], [], [], [], [])"
227,2022-09-22,"([], [], [], [], [], [])"
228,2022-10-12,"([일부 위원은 경제전망 시 금리지표로 어떠한 시장금리를 사용하는지, 그리고 금번..."


In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230 entries, 0 to 229
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   date     230 non-null    object
 1   title    230 non-null    object
 2   content  230 non-null    object
dtypes: object(3)
memory usage: 5.5+ KB
